In [ ]:
def foo(x, y):
#     print(x.shape)
#     print(y.shape)
    x, y = x[:, None, :], y[None, :, :]
    x, y = np.repeat(x, x.shape[0], axis=-2), np.repeat(y, y.shape[1], axis=-3) 
#     print("=====1")
#     print(x.shape)
#     print(y.shape)
#     print("=====2")
    return euclidean(x, y)

def foo2(x, y):
    return euclidean(x[:, None, :], y[None, :, :])

In [ ]:
def pairwise(x):
    return da.blockwise(
        # Lambda wraps reshape for broadcast
        euclidean, 'jk', 
        x, 'ji', x, 'ki', 
        dtype='float64',
        concatenate=True
    )

In [ ]:
cupy.asarray(pairwise(x).compute(scheduler="single-threaded"))

In [ ]:
pairwise(x).compute(scheduler="single-threaded")

In [ ]:
cs = cupy.random.RandomState(0)
x = cs.randint(0, 3, size=(2, 10)).astype('f8')
y = cs.randint(0, 3, size=(2, 10)).astype('f8')

In [ ]:
x, y

In [ ]:
foo2(x, y)

In [ ]:
import math
import numpy as np
import cupy

from numba import guvectorize

@guvectorize(  # type: ignore
    [
        "void(float32[:], float32[:], float32[:])",
        "void(float64[:], float64[:], float64[:])",
        "void(int8[:], int8[:], float64[:])",
    ],
    "(n),(n)->()", target='cuda'
)
def euclidean(x, y, out) -> None:
    square_sum = 0.0
    m = x.shape[0]
    # Ignore missing values
    for i in range(m):
        if x[i] >= 0 and y[i] >= 0:
            square_sum += (x[i] - y[i]) ** 2
    out[0] = math.sqrt(square_sum)

def foo1(x, y):
    x, y = x[:, None, :], y[None, :, :]
    x, y = np.repeat(x, x.shape[0], axis=-2), np.repeat(y, y.shape[1], axis=-3) 
    return euclidean(x, y)

def foo2(x, y):
    return euclidean(x[:, None, :], y[None, :, :])

cs = cupy.random.RandomState(0)
x = cs.randint(0, 3, size=(2, 10)).astype('f8')
y = cs.randint(0, 3, size=(2, 10)).astype('f8')

cupy.asarray(foo1(x, y))
# cupy.asarray(foo2(x, y))

In [ ]:
import math
import numpy as np
import cupy

from numba import guvectorize

@guvectorize(  # type: ignore
    [
        "void(float32[:], float32[:], float32[:])",
        "void(float64[:], float64[:], float64[:])",
        "void(int8[:], int8[:], float64[:])",
    ],
    "(n),(n)->()", target='cuda', fastmath=True
)
def g(x, y, out) -> None:
    diff = 0.0
    for i in range(x.shape[0]):
        diff += x[i] - y[i]
    out[0] = diff

def foo_working(x, y):
    x, y = x[:, None, :], y[None, :, :]
    x, y = np.repeat(x, x.shape[0], axis=-2), np.repeat(y, y.shape[1], axis=-3) 
    return g(x, y)

def foo_not_working(x, y):
    return g(x[:, None, :], y[None, :, :])

cs = cupy.random.RandomState(0)
x = cs.randint(0, 3, size=(2, 10)).astype('f8')
y = cs.randint(0, 3, size=(2, 10)).astype('f8')

cupy.asarray(foo_working(x, y))
# cupy.asarray(foo_not_working(x, y))

In [ ]:
def foo2(x, y):
    return euclidean(x[:, None, :], y[None, :, :])

In [ ]:
cs = cupy.random.RandomState(0)
x = cs.randint(0, 3, size=(2, 10)).astype('f8')
y = cs.randint(0, 3, size=(2, 10)).astype('f8')
cupy.asarray(foo2(x, y))

In [ ]:
from sgkit.distance.api import pairwise_distance
rs = da.random.RandomState(RandomState=cupy.random.RandomState)
x = rs.randint(0, 3, size=(10000, 10000), dtype='i1', chunks=(2000, -1))
x

In [ ]:
%%time
with ProgressBar(), ResourceProfiler() as prof:
    out = pairwise_distance(x.rechunk((None, -1)), metric='euclidean')
prof.visualize()

In [ ]:
%%time
with ProgressBar(), ResourceProfiler() as prof:
    out = pairwise_distance(x.rechunk((None, -1)), metric='euclidean')
prof.visualize()

In [ ]:
type(x)

In [ ]:
import cupy
import dask.array as da

# generate chunked dask arrays of mamy numpy random arrays
rs = da.random.RandomState()
x = rs.normal(10, 1, size=(100000, 100000), chunks=(10000, 10000))

In [ ]:
%%time
with ProgressBar(), ResourceProfiler() as prof:
    (x + 1)[::2, ::2].sum().compute(scheduler='threads')
prof.visualize()

In [ ]:
rs = da.random.RandomState(RandomState=cupy.random.RandomState)  # <-- we specify cupy here
x = rs.normal(10, 1, size=(50000, 50000), chunks=(10000, 10000))

In [ ]:
%%time
with ProgressBar(), ResourceProfiler() as prof:
    (x + 1)[::2, ::2].sum().compute(scheduler='threads')
prof.visualize()